# Flipping, adding keywords, and calibration of light frames

In [1]:
from astropy.coordinates import SkyCoord
import astropy.units as u
import glob,os

## Keeler 24" Planewave Specs

In [3]:
FOCALLEN = 3962.3999023437500 #Focal length of telescope in mm
PIXELSIZE = 9 #pixel size in microns (from https://www.astroshop.eu/astronomical-cameras/sbig-camera-stx-16803-mono/p,53788#specifications)

In [ ]:
target_coords_wcs = ("00:28:12.944", "+42:03:40.95")
c=SkyCoord(target_coords_wcs[0],target_coords_wcs[1],frame='icrs',unit=(u.hourangle,u.degree))
ra=c.ra.degree; dec=c.dec.degree
print("ra:", ra, "dec:",c.dec.degree)
epoch=2000.0
pixscale=206.265*(PIXELSIZE/FOCALLEN)  # in arcsec, formula from https://www.icrar.org/wp-content/uploads/2021/11/Resolving-the-night-sky-updated-2021.pdf
print(pixscale)

ra: 7.053933333333333 dec: 42.061375
0.4685001629699094


## Adding keywords to light frames, initial references...

## Gain and Readout Noise

In [15]:
from astropy.io import fits
import numpy as np
import glob

def get_gain_readout_noise(directory:str, bias_indicator:str, flat_indicator:str) -> list:
    bias_list = glob.glob(directory + f"*{bias_indicator}*")
    flat_list = glob.glob(directory + f"*{flat_indicator}*")
    print(len(bias_list))
    print(len(flat_list))
    bias_data = []
    flat_data = []
    
    for bias_file in bias_list:
        data = fits.getdata(bias_file)[1500-256:1500+256,1500-256:1500+256]
        bias_data.append(data)
    for flat_file in flat_list:
        data = fits.getdata(flat_file)[1500-256:1500+256,1500-256:1500+256]
        flat_data.append(data)
    
    bias_combined = np.median(bias_data, axis=0)
    flat_combined = np.median(flat_data, axis=0)
    
    # Calculate gain and read noise
    mean_flat = np.mean(flat_combined)
    mean_bias = np.mean(bias_combined)
    std_flat = np.std(flat_combined)
    std_bias = np.std(bias_combined)

    gain = (mean_flat - mean_bias) / (std_flat**2 - std_bias**2)
    readnoise = gain * std_bias / np.sqrt(2)  
      
    print(f"gain: {gain}, readoout: {readnoise}")
    return [gain, readnoise]

get_gain_readout_noise("/Users/spencerfreeman/Desktop/PersonalCS/CurrentPipeline/test_input/", "bias", "frp")

11
9
gain: 0.24275960202545627, readoout: 0.6125354503773921


[0.24275960202545627, 0.6125354503773921]